In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from definitions import ROOT_DIR, main_activity_label_classes
from trainTestValSplit import getTrainTestValSplit
from preProcessing import preProcess
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import f1_score

In [30]:
train_concat, test_concat, val_concat = getTrainTestValSplit(
    _dataset_path=f'{ROOT_DIR}/dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels'
)


KeyboardInterrupt



In [61]:
train_x, train_y = preProcess(train_concat)
test_x, test_y = preProcess(test_concat)
val_x, val_y = preProcess(val_concat)

In [62]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)
val_x = val_x.fillna(0)

In [63]:
n_neurons: int = 1024
window_size: int = 5
epochs: int = 20
n_y: int = len(main_activity_label_classes)
batch_size: int = 1024
n_features: int = len(train_x.columns)

In [64]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()

In [65]:
# Source: https://stackoverflow.com/questions/43114460/is-there-a-way-to-reshape-an-array-that-does-not-maintain-the-original-size-or
def reshape_and_truncate(arr, shape):
    desired_size_factor = np.prod([n for n in shape if n != -1])
    if -1 in shape:  # implicit array size
        desired_size = arr.size // desired_size_factor * desired_size_factor
    else:
        desired_size = desired_size_factor
    return arr.flat[:desired_size].reshape(shape)

In [66]:
def encodeTarget(y):
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    return np_utils.to_categorical(y)

In [67]:
train_y = encodeTarget(train_y)
test_y = encodeTarget(test_y)
val_y = encodeTarget(val_y)

In [68]:
train_x = reshape_and_truncate(train_x, (-1, window_size, n_features))
train_y = reshape_and_truncate(train_y, (-1, window_size, len(main_activity_label_classes)))

test_x = reshape_and_truncate(test_x, (-1, window_size, n_features))
test_y = reshape_and_truncate(test_y, (-1, window_size, len(main_activity_label_classes)))

val_x = reshape_and_truncate(val_x, (-1, window_size, n_features))
val_y = reshape_and_truncate(val_y, (-1, window_size, len(main_activity_label_classes)))

In [84]:
with tf.device('/cpu:0'):
    model = tf.keras.Sequential()

    layer = tf.keras.layers.Normalization(axis=-1)
    layer.adapt(train_x)
    model.add(layer)
    model.add(tf.keras.layers.LSTM(150, return_sequences=True))
    model.add(tf.keras.layers.Dropout(.2))
    model.add(tf.keras.layers.LSTM(65, return_sequences=True))
    model.add(tf.keras.layers.Dropout(.2))
    model.add(tf.keras.layers.Dense(n_y, activation=tf.keras.activations.softmax))

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['acc'])
    # model.summary()
    model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y), epochs=10, batch_size=1024)
    predict = model.predict(test_x)

Epoch 1/10
54/54 [==============================] - 12s 173ms/step - loss: 1.2078 - acc: 0.6098 - val_loss: 0.9641 - val_acc: 0.6955
Epoch 2/10
54/54 [==============================] - 9s 158ms/step - loss: 0.6953 - acc: 0.7768 - val_loss: 0.9002 - val_acc: 0.7002
Epoch 3/10
54/54 [==============================] - 9s 160ms/step - loss: 0.5643 - acc: 0.8132 - val_loss: 0.8669 - val_acc: 0.7152
Epoch 4/10
54/54 [==============================] - 9s 172ms/step - loss: 0.4853 - acc: 0.8350 - val_loss: 0.8567 - val_acc: 0.7190
Epoch 5/10
54/54 [==============================] - 9s 172ms/step - loss: 0.4311 - acc: 0.8517 - val_loss: 0.8958 - val_acc: 0.7151
Epoch 6/10
54/54 [==============================] - 10s 187ms/step - loss: 0.3966 - acc: 0.8627 - val_loss: 0.9353 - val_acc: 0.7062
Epoch 7/10
54/54 [==============================] - 10s 176ms/step - loss: 0.3696 - acc: 0.8721 - val_loss: 0.9300 - val_acc: 0.7127
Epoch 8/10
54/54 [==============================] - 10s 177ms/step - loss

In [40]:
def get_labels(windows):
    predicted_labels = []

    for p in windows:
        df = pd.DataFrame(p)
        probabilities = []

        for col in df.columns:
            probabilities.append(df[col].mean())
        predicted_labels.append(np.argmax(probabilities))
    return predicted_labels

In [85]:
gold_labels = get_labels(test_y)
predicted_labels = get_labels(predict)

In [86]:
f1_score(gold_labels, predicted_labels, average='weighted')

0.752553632060654

In [87]:
tf.math.confusion_matrix(gold_labels, predicted_labels)

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[2860,    4,   19,   54,    3,    0,    0],
       [   8, 2737,  507,   55,    4,    0,    0],
       [  57,  531, 4542,  388,  109,    0,    4],
       [  17,  151,  718,  323,   80,    1,    7],
       [  10,   11,  362,  151,  370,    0,    0],
       [   0,   11,   23,   22,   16,    8,    0],
       [  12,    5,   31,    6,    7,    0,   45]])>